In [1]:
import os
import gc

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import PatchTSTForPrediction
from datasets import Dataset

2025-11-09 15:00:10.293576: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = "coin"

output_dir = "saved_models"
log_dir = os.path.join('logstf', data)

loss_name = "mse"

num_train_epochs = 2000
model_num = 1
model_path = "./saved_models"
learning_rate = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [4]:
def array_to_dataset(X, y):
    X, y = torch.tensor(X), torch.tensor(y)
    X = X.reshape(-1, X.shape[1], 1)
    y = y.reshape(-1, y.shape[1], 1)

    dataset = torch.utils.data.TensorDataset(X, y)

    return dataset

train_dataset = array_to_dataset(target_X, target_y)
val_dataset = array_to_dataset(target_X_val, target_y_val)
test_dataset = array_to_dataset(test_X, test_y)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)

In [5]:
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, dropout = 0.05, max_len = 5000, **kwargs):
        super().__init__()
        self.dropout = torch.nn.Dropout(p = dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)) * (-np.log(10000.0) / d_model)

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(torch.tensor(position * div_term))
        pe[:, 1::2] = torch.cos(torch.tensor(position * div_term))
        pe = pe.unsqueeze(0)    ## (1, max_len, d_model)
        
        self.register_buffer("pe", pe)  ## 불변값. 학습되지 않음. tf.constant

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [6]:
class TransformerHead(torch.nn.Module):
    def __init__(self, d_model, nlayers, nhead, dropout, iw, ow, input_dim):
        super().__init__()
        
        self.input_adapter = torch.nn.Sequential(
            torch.nn.Linear(input_dim, d_model // 2),
            torch.nn.ReLU(),
            torch.nn.Linear(d_model // 2, d_model),
            torch.nn.ReLU()
        )

        self.pos_encoding = PositionalEncoding(d_model, dropout)    ## patch만 position encoding

        self.layers = torch.nn.ModuleList([
            torch.nn.ModuleDict({
                "attn": torch.nn.MultiheadAttention(embed_dim = d_model, num_heads = nhead, dropout = dropout, batch_first = True),
                "norm1": torch.nn.LayerNorm(d_model, eps = 1e-6),
                "ffn1": torch.nn.Linear(d_model, d_model),
                "relu": torch.nn.ReLU(),
                "ffn2": torch.nn.Linear(d_model, d_model),
                "norm2": torch.nn.LayerNorm(d_model, eps = 1e-6)
            }) for _ in range(nlayers)
        ])

        self.outlayer = torch.nn.Sequential(
            torch.nn.Linear(d_model, d_model // 2),         ## (B, 7, 42)
            torch.nn.ReLU(),
            torch.nn.Flatten(),                             ## (B, 7*42 = 294). squeeze 역할
            torch.nn.Linear(iw * (d_model // 2), 128),      ## (B, 128)
            torch.nn.ReLU(),
            torch.nn.Linear(128, ow) ## (B, 24)
        )

    def forward(self, x):
        ## x : (B, 1, 7, 256)
        x = x.squeeze(1)            ## (B, 7, 256)
        x = self.input_adapter(x)   ## (B, 7, 84)
        x = self.pos_encoding(x)    ## (B, 7, 84)

        for layer in self.layers:
            attn_output, _ = layer["attn"](x, x, x)
            x = layer["norm1"](x + attn_output)

            ffn_output = layer["relu"](layer["ffn1"](x))
            ffn_output = layer["ffn2"](ffn_output)
            x = layer["norm2"](x + ffn_output)

        outputs = self.outlayer(x)
        outputs = outputs.unsqueeze(2)

        return outputs

In [7]:
for k in range(1, model_num+1):
    current_path = os.path.join(model_path, f"model_{loss_name}_{k}.pth")

    model_instance = PatchTSTForPrediction.from_pretrained(os.path.join(model_path, "PatchTSTBackbone")).to(device)
    model_instance.load_state_dict(torch.load(current_path))
    
    model_instance.head.flatten = torch.nn.Identity()
    model_instance.head.projection = TransformerHead(
        d_model = 84, nlayers = 2, nhead = 2, dropout = 0.2,
        iw = 7, ow = target_y.shape[1], input_dim = 256
    )
    model_instance.dropout = torch.nn.Identity()
    model_instance.to(device)

/tmp/ipykernel_1640616/4015960496.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pe[:, 0::2] = torch.sin(torch.tensor(position * div_term))
/tmp/ipykernel_1640616/4015960496.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pe[:, 1::2] = torch.cos(torch.tensor(position * div_term))


In [8]:
## custom loss function
def SMAPE(yhat, y):
    numerator = 100*torch.abs(y - yhat)
    denominator = (torch.abs(y) + torch.abs(yhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

def MAPE(y_pred, y_true, epsilon=1e-7):
    denominator = torch.clamp(torch.abs(y_true), min=epsilon)       ## 분모에 0이 들어오는 것을 방지
    abs_percent_error = torch.abs((y_true - y_pred) / denominator)

    return torch.mean(100. * abs_percent_error)


class MASE(torch.nn.Module):
    def __init__(self, training_data, period = 1):
        super().__init__()
        ## 원본 코드 구현, 사실상 MAE와 동일, 잘못 짜여진 코드, 일단은 하던대로 할 것.
        self.scale = torch.mean(torch.abs(torch.tensor(training_data[period:] - training_data[:-period])))
    
    def forward(self, yhat, y):
        error = torch.abs(y - yhat)
        return torch.mean(error) / self.scale

In [9]:
optimizer = torch.optim.Adam(model_instance.parameters(), lr = learning_rate)
log_data = []

if loss_name == "mse":
    loss_fn = torch.nn.MSELoss()
elif loss_name == "mae":
    loss_fn = torch.nn.L1Loss()
elif loss_name == "SMAPE":
    loss_fn = SMAPE
elif loss_name == "mape":
    loss_fn = MAPE
elif loss_name == "MASE":
    loss_fn = MASE(target_y, target_y.shape[1])
else:
    raise Exception("Your loss name is not valid.")

## early stopping
PATIENCE = 10
best_val_loss = np.inf
patience_counter = 0

for epoc in range(num_train_epochs):
    model_instance.train()

    total_train_loss = 0

    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        yhat = model_instance(X).prediction_outputs
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()*X.shape[0]

    avg_train_loss = total_train_loss/len(train_dataloader.dataset)

    model_instance.eval()

    with torch.no_grad():
        yys = []
        yyhats = []

        for XX, yy in val_dataloader:
            XX = XX.to(device)
            yys.append(yy.to(device))
            yyhats.append(model_instance(XX).prediction_outputs)

        yyhat = torch.concat(yyhats)
        yy = torch.concat(yys)

        val_loss = loss_fn(yyhat, yy).item()

    print(f"Epoch {epoc+1}/{num_train_epochs} | Train Loss: {avg_train_loss:.6f}\t\t Val Loss: {val_loss:.6f}")

    log_data.append({"epoch": epoc, "loss": avg_train_loss, "eval_loss": val_loss})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state_dict = model_instance.state_dict()   ## 저장 없이 결과물만 산출...
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        break

/root/anaconda3/envs/TF/lib/python3.12/site-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([8, 24, 1])) that is different to the input size (torch.Size([8, 1, 24])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/root/anaconda3/envs/TF/lib/python3.12/site-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([2, 24, 1])) that is different to the input size (torch.Size([2, 1, 24])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/root/anaconda3/envs/TF/lib/python3.12/site-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([145, 24, 1])) that is different to the input size (torch.Size([145, 1, 24])). This will likely lead to incorrect results due to broadcast

Epoch 1/2000 | Train Loss: 376.306846		 Val Loss: 30.860167
Epoch 2/2000 | Train Loss: 375.512298		 Val Loss: 30.823681
Epoch 3/2000 | Train Loss: 375.035959		 Val Loss: 30.772827
Epoch 4/2000 | Train Loss: 374.433940		 Val Loss: 30.725220
Epoch 5/2000 | Train Loss: 375.171625		 Val Loss: 30.680365
Epoch 6/2000 | Train Loss: 374.912508		 Val Loss: 30.648054


KeyboardInterrupt: 

In [ ]:
model_instance.eval()

with torch.no_grad():
    yys = []
    yyhats = []

    for XX, yy in test_dataloader:
        XX = XX.to(device)
        yys.append(yy.to(device))
        yyhats.append(model_instance(XX).prediction_outputs)

    yyhat = torch.concat(yyhats)
    yy = torch.concat(yys)

    test_loss = loss_fn(yyhat, yy)

In [ ]:
mseLoss = torch.nn.MSELoss()
maeLoss = torch.nn.L1Loss()

def smape(yy, yyhat):
    numerator = 100*abs(yy - yyhat)
    denominator = (abs(yy) + abs(yyhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

print(f"test RMSE: {torch.sqrt(mseLoss(yyhat, yy))}")
print(f"test MAE: {maeLoss(yyhat, yy)}")
print(f"test SMAPE: {smape(yy, yyhat)}")

test RMSE: 3.929018497467041
test MAE: 1.772931694984436
test SMAPE: 2.4976139068603516
